# Lyne

## Operation

```python
@Op(I.attr) >> I.target
def cust_print(value, def_val):
    print(value, def_val)
    return value

pipe = (
    #Custom wrapper
    Op[print](I.attr)
    
    #Default operation
    | cust_print
    
    #Adds args
    | cust_print(def_val='test')
    
    #Overrides args
    | cust_print.using(I.other, def_val='test')
)
```

## Proxy
```python
I.attr * 255
I['field']
I[lambda I: max(I.value, 0)]
```
---
```
Process batch:
S[I, I] => func(S) -> S => S[I, I]

Aggregate:
S[I, I] => func(S) -> I => I

Process item:
I => func(I) -> I => I

Propagate, distribute
I => func(I) -> S => S[I, I]
```

In [ ]:
import unittest
from lyne import *


class TestProxy(unittest.TestCase):
    def _assertTrueEqual(self, first, second, msg=None):
        return self.assertIs(first == second, True, msg)
    
    def setUp(self):
        self.I = ItemProxy()
        self.S = StreamProxy()
        self.O = OutputProxy()
    
    def test_unique_instance(self):
        self.assertIsNot(self.I, self.I.attr)
        self.assertIsNot(self.I.attr, self.I.attr)
        
    def test_instance(self):
        with self.assertRaises(TypeError):
            obj = AbstractProxy()
        with self.assertRaises(TypeError):
            obj = AssignableProxy()
        with self.assertRaises(TypeError):
            obj = InputProxy()
        self.assertIsInstance(self.I.attr, ItemProxy)
        self.assertIsInstance(self.S.attr, StreamProxy)
        self.assertIsInstance(self.I, InputProxy)
        self.assertIsInstance(self.S, InputProxy)
        self.assertNotIsInstance(self.O, InputProxy)
        self.assertIsInstance(self.O, OutputProxy)
    
    def test_assignable(self):
        self.assertIsInstance(self.I.item, AssignableProxy)
        self.assertNotIsInstance(self.I * 3, AssignableProxy)
        self.assertNotIsInstance(self.O, AssignableProxy)
        self.assertIsNotNone(hash(self.I.item))
    
    def test_lambda(self):
        l = self.I[lambda item: item['test']]
        self.assertIsInstance(l, LambdaProxy)
        
        item = {'test': 123}
        result = Proxy.get_value(l, item)
        self._assertTrueEqual(result, 123)
        
        l = self.I[lambda item: str(item['test'])]
        result = Proxy.get_value(l, item)
        self._assertTrueEqual(result, '123')
        
    def test_get_value(self):
        item = {'test': 123}
        result = Proxy.get_value(self.I['test'], item)
        self._assertTrueEqual(result, 123)
        with self.assertRaises(Exception):
            Proxy.get_value(self.I['invalid_key'], item)

        proxy = ItemProxy()['test']
        proxy += 3
        result = Proxy.get_value(proxy, item)
        self._assertTrueEqual(result, 126)
        
        proxy = 10  * self.I['val_a'] + self.I['val_b']
        item = {'val_a': 2, 'val_b': 10}
        result = Proxy.get_value(proxy, item)
        self._assertTrueEqual(result, 30)
        

class TestItem(unittest.TestCase):
    
    def setUp(self):
        self.item = Item(test='test', value=123, deep_test=[])
        
    def __deepcopy__(self, memo=None):
        import copy
        return self.__class__(**copy.deepcopy(dict(self), memo=memo))        
    
    def test_values(self):
        self.assertEqual(self.item.test, 'test')
        self.assertEqual(self.item['test'], 'test')
        self.assertEqual(self.item.value, 123)
        self.assertIsNone(self.item.non_existing)
        del self.item.value
        del self.item.non_existing
        
        self.item.non_existing = 3
        self.assertNotIn('non_existing', self.item.__dict__)

    def test_copy(self):
        import copy
        
        other = copy.copy(self.item)
        self.assertEqual(self.item.value, other.value)
        self.assertIs(self.item.deep_test, other.deep_test)

        other = copy.deepcopy(self.item)
        self.assertEqual(self.item.value, other.value)
        self.assertIsNot(self.item.deep_test, other.deep_test)


class TestStream(unittest.TestCase):
    def test_stream(self):
        l = [0, 1, 2, 3]
        g = (i for i in range(4))
        s_l = Stream(l)
        s_g = Stream(g)
        self.assertEqual(list(s_l), l)
        self.assertEqual(list(s_g), l)
        with self.assertRaises(StopIteration):
            next(g)
        
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)